# Main Notebook

This notebook represents the main notebook in which we showcase our main line of experiments. \
Please make sure to activate the environment before running this notebook.

In [37]:
# Standard imports
import os
import torch

script_dir = os.path.abspath('')

# Set this to false to see wandb output
os.environ["WANDB_SILENT"] = "true"

# Set this to "online" for loggin metrics to cloud
!wandb offline 

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


### About this notebook

This notebook showcases the main experiments that we base our paper on. 
Our experimental setting is comprised of two main sets of experiments, namely:

**i)** exploring the benefits of incorporating implicit topology in the EGNN framework, and \
**ii)** examining the trade-off between model complexity and the contribution of the topological information

We acknowledge that not anybody might have access to the required computational resources to train each of the models we tested, and thus we provide the saved model weights in the HuggingFace repository [here](https://huggingface.co/datasets/lucapantea/egnn-lspe/tree/main). We thus load each of the model weights saved during training in the following cell.

In [38]:
# Create saved_models dir
saved_models_dir = os.path.join(os.path.dirname(script_dir), 'saved_models')
if not os.path.exists(saved_models_dir):
    os.makedirs(saved_models_dir)

# Load model weights from hugging face
saved_models_dir_git = r'"{}"'.format(saved_models_dir)
if os.path.exists(saved_models_dir) and len(os.listdir(saved_models_dir)) == 0:
    !git clone https://huggingface.co/datasets/lucapantea/egnn-lspe {saved_models_dir_git}
else:
    print('Model weights already initialized.')

Model weights already initialized.


## TODO
The cells below show how to run all experiments by only using evaluate. The model weight path is given through the `evaluate` argument. All the other arguments for selecting the model should be provided manually.

In [39]:
# first
!python -W ignore ../main.py --evaluate "mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt" \
                            --dataset "qm9_fc" \
                            --pe "nope" \
                            --pe_dim 0 \
                            --include_dist 

Namespace(evaluate='mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt', model='mpnn', dataset='qm9_fc', pe='nope', pe_dim=0, lspe=False, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=True, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 743809

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:22<00:00,  4.66it/s]

Test MAE: 0.498
Evaluation finished. Exiting...


In [40]:
# second
!python -W ignore ../main.py --evaluate "mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt" \
                            --dataset "qm9_fc" \
                            --pe "nope" \
                            --include_dist


Namespace(evaluate='mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt', model='mpnn', dataset='qm9_fc', pe='nope', pe_dim=24, lspe=False, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=True, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 743809

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_fc_nope_no-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:22<00:00,  4.64it/s]

Test MAE: 0.113
Evaluation finished. Exiting...


In [42]:
# third
!python -W ignore ../main.py --evaluate "mpnn_qm9_nope_no-lspe_no-dist_no-reduced_yes-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt" \
                            --dataset "qm9" \
                            --pe "nope" \
                            --update_with_pe

Namespace(evaluate='mpnn_qm9_nope_no-lspe_no-dist_no-reduced_yes-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt', model='mpnn', dataset='qm9', pe='nope', pe_dim=24, lspe=False, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=False, reduced=False, update_with_pe=True)

MPS available. Setting device to MPS.
Number of parameters: 742913

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_nope_no-lspe_no-dist_no-reduced_yes-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:09<00:00, 10.98it/s]

Test MAE: 1.452
Evaluation finished. Exiting...


In [45]:
# fourth
!python -W ignore ../main.py --evaluate "mpnn_qm9_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt" \
                            --dataset "qm9" \
                            --pe "nope" \
                            --include_dist

Namespace(evaluate='mpnn_qm9_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt', model='mpnn', dataset='qm9', pe='nope', pe_dim=24, lspe=False, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=True, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 743809

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_nope_no-lspe_yes-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:12<00:00,  8.44it/s]

Test MAE: 0.251
Evaluation finished. Exiting...


In [48]:
# fifth
!python -W ignore ../main.py --evaluate "mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt" \
                            --dataset "qm9" \
                            --pe "rw" \
                            --pe_dim 24 \
                            --lspe

Namespace(evaluate='mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt', model='mpnn', dataset='qm9', pe='rw', pe_dim=24, lspe=True, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=False, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 1670273

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:14<00:00,  7.00it/s]

Test MAE: 0.214
Evaluation finished. Exiting...


In [50]:
# sixth
!python -W ignore ../main.py --evaluate "mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt" \
                            --dataset "qm9" \
                            --pe "rw" \
                            --pe_dim 24 \
                            --lspe \
                            --seed 42

Namespace(evaluate='mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt', model='mpnn', dataset='qm9', pe='rw', pe_dim=24, lspe=True, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=False, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 1670273

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:15<00:00,  6.91it/s]

Test MAE: 0.215
Evaluation finished. Exiting...


In [52]:
# seventh
!python -W ignore ../main.py --evaluate "mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt" \
                            --dataset "qm9" \
                            --pe "rw" \
                            --pe_dim 24 \
                            --lspe \
                            --seed 420 

Namespace(evaluate='mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt', model='mpnn', dataset='qm9', pe='rw', pe_dim=24, lspe=True, seed=420, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=False, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 1670273

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_rw24_yes-lspe_no-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:15<00:00,  6.86it/s]

Test MAE: 0.223
Evaluation finished. Exiting...


In [54]:
# eith
!python -W ignore ../main.py --evaluate "mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt" \
                            --dataset "qm9" \
                            --pe "rw" \
                            --pe_dim 24 \
                            --lspe \
                            --include_dist

Namespace(evaluate='mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt', model='mpnn', dataset='qm9', pe='rw', pe_dim=24, lspe=True, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=True, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 1672065

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-42.pt...

Beginning evaluation...
100%|█████████████████████████████████████████| 105/105 [00:18<00:00,  5.61it/s]

Test MAE: 0.216
Evaluation finished. Exiting...


In [55]:
# ninth
!python ../main.py --evaluate "mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt" \
                    --dataset "qm9" \
                    --pe "rw" \
                    --pe_dim 24 \
                    --lspe \
                    --include_dist

Namespace(evaluate='mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt', model='mpnn', dataset='qm9', pe='rw', pe_dim=24, lspe=True, seed=42, epochs=1000, batch_size=96, learning_rate=0.0005, weight_decay=1e-16, in_channels=11, hidden_channels=128, num_layers=7, out_channels=1, include_dist=True, reduced=False, update_with_pe=False)

MPS available. Setting device to MPS.
Number of parameters: 1672065

Loading model with weights stored at /Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/demos/../saved_models/mpnn_qm9_rw24_yes-lspe_yes-dist_no-reduced_no-update_with_pe_epochs-1000_num_layers-7_in_c-11_h_c-128_o_c-1_bs-96_lr-0.0005_seed-420.pt...

Beginning evaluation...
  0%|                                                   | 0/105 [00:00<?, ?it/s]/Users/luca/Documents/Masters/Deep Learning 2/LSPE-EGNN/models/mpnn.py:94: UserWarning: The operator 'aten::linalg_vector_norm' is not currently suppor

In [4]:
# Experiment 2: Details, formulas, and running (either train or evaluate directly using model weights)
# Use argument --evaluate to run experiments with the saved model weights

In [5]:
# Extra Studies?